In [1]:
import pandas as pd 
import requests
from bs4 import BeautifulSoup
import csv
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.llms import Ollama
from langchain_experimental.agents import create_pandas_dataframe_agent
from dotenv import load_dotenv
import os

In [2]:
url="https://www.harighotra.co.uk/indian-recipes/time/indian-recipes-in-30-60-minutes/view-all"
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')


In [3]:
articles = soup.find_all('article')

In [4]:
recipe=[]
for a in articles:
    name_tag= a.find("h3")
    time_tag= a.find('h4',attrs={"content":True})
    if name_tag and time_tag:
        name = name_tag.text.strip()
        time = time_tag.text.strip()
        recipe.append([name, time])

In [5]:
with open('data.csv','w',newline='',encoding='utf-8')as file:
    writer=csv.writer(file)
    writer.writerow(['name','time'])
    writer.writerows(recipe)

In [6]:
df=pd.read_csv('data.csv')

In [7]:
df['time'].head(10)

0            1 hr
1         50 mins
2            1 hr
3         30 mins
4         50 mins
5    1 hr 20 mins
6         40 mins
7         50 mins
8         50 mins
9         40 mins
Name: time, dtype: object

In [8]:
def time_min(time_str):
    time_str=str(time_str).lower().strip()
    min=0
    if 'hr' in time_str:
        hr_part=time_str.split('hr')[0].strip()
        if hr_part.isdigit():
            min+=int(hr_part)*60
    if 'min' in time_str:
        min_part=''.join(filter(str.isdigit,time_str.split('hr')[-1]))
        if min_part:
            min+=int(min_part)
    return min

In [9]:
df['time_min']=df['time'].apply(time_min)

In [10]:
df.head(10)

,name,time,time_min
0,Pistachio chicken,1 hr,60
1,Turkey Biryani,50 mins,50
2,Healthy Chicken Korma,1 hr,60
3,Brown Lentil Dhal,30 mins,30
4,Thari Wala Chicken,50 mins,50
5,Lamb Dhansak,1 hr 20 mins,80
6,Chicken Pasanda,40 mins,40
7,Achaari Bangun,50 mins,50
8,Aloo Bangun,50 mins,50
9,Bombay Aloo,40 mins,40


In [11]:
langchain_key = os.getenv("LANGCHAIN_API_KEY")
langsmith_key = os.getenv("LANGSMITH_API_KEY")
langsmith_project = os.getenv("LANGSMITH_PROJECT")

In [ ]:
load_dotenv()
# os.environ["LANGCHAIN_API_KEY"] = langchain_key
os.environ["LANGSMITH_API_KEY"] = langsmith_key
print(langsmith_key)
os.environ["LANGCHAIN_TRACKING_V2"] = "true"
LANGSMITH_PROJECT="meal_bot"

TypeError: str expected, not NoneType

In [ ]:

# Connect to Gemma (via Ollama)
llm = Ollama(model='gemma3')
# Create LangChain agent that can read the dataframe
agent = create_pandas_dataframe_agent(llm, df, allow_dangerous_code=True, verbose=True)

# Ask your question
response = agent.run(
    "Pick 3 different meals for 3 days: one breakfast, one lunch, one dinner. "
    "Choose randomly and return them in a table with Meal, Dish Name, and Time (mins)."
)

print(response)




> Entering new AgentExecutor chain...
Thought: I need to randomly select three meals from the dataframe and create a table with the specified columns. I can use the `numpy.random.choice` function to select rows from the dataframe, and then extract the relevant columns.
Action: python_repl_ast
Action Input: ```python
import numpy as np
np.random.seed(42)
indices = np.random.choice(df.index, size=3)
result = df.loc[indices]
print(result[['name', 'time_min']])
```                       name  time_min
102  Sweet Coconut Pancakes        40
51         Hot Sticky Wings        40
92            Shammi Kebabs        40
The output of the previous action is not what I expected. I need to generate 3 meals randomly. I will use `numpy.random.choice` again but this time I will seed the random number generator to ensure reproducibility.
Action: python_repl_ast
Action Input: ```python
import numpy as np
np.random.seed(42)
indices = np.random.choice(df.index, size=3)
result = df.loc[indices]
print(resu